# Instructions

In this challenge, you'll try to predict the severity of car accidents, based on features collected from after-crash police investigation

This [Kaggle challenge](https://www.kaggle.com/c/accident-severity) comprises of 1,000,000 accidents report, split into multiple `.csv` files.

**The goal of the model is to predict the severity of car accidents**. The target variable is called `grav` (for 'gravity') in the file `users.csv`. This variable has four levels, but in this challenge, we'll convert it to a binary classification problem. We will:
- Load data into pandas
- Create a single DataFrame for our problem, where each row is a user involved in an accident
- Extract the features you think would be relevant to predict its severity
- Build a data pipeline that gives you a baseline model
- Then, iterate on the different phase and try to get the best model! 

🔥 **Today is a special challenge** :
- You will send your best score to your batch slack channel!
- The winner will present its notebook to the class during the recap session at 5pm 💪

---
⚠️ **Good practices to follow for large exploratory notebooks**
- Build your Notebook linearily so that it can always be run from top to bottom without any errors
- Clean the outputs of your cells that are not needed
- Clean your variables in memory when you don't need them (especially when they are very large). You can use the python built-in function `del`, or the the **Jupyter nbextentions** `variable_inspector`
- Make heavy use of `table_of_content` and `collapsable_headings` 

# Data sourcing

Let's get started! The data we want to use is from the `csv` files in `/data/data_training`

## Loading data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cara = pd.read_csv("data/data_training/caracteristics.csv", encoding="ISO-8859-1")
users = pd.read_csv("data/data_training/users.csv", encoding="ISO-8859-1")
places = pd.read_csv("data/data_training/places.csv", encoding="ISO-8859-1")
vehicles = pd.read_csv("data/data_training/vehicles.csv", encoding="ISO-8859-1")

/home/florent/.pyenv/versions/3.8.5/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


❓ Explore the different tables, and the different variables using `challenge_variable.md`, which provides a description of features. More details can be found [here](https://www.data.gouv.fr/fr/datasets/r/8d4df329-bbbb-434c-9f1f-596d78ad529f) if needed, or in the [Kaggle](https://www.kaggle.com/ahmedlahlou/accidents-in-france-from-2005-to-2016/discussion) discussion channel. Understand

## Merge datasets

❓ We will create one single dataset where each row should represent a `user` in a car, by merging the data from the different files dataset.  
**Take some time to think about how you would do it yourself**, and only then, read carefully through the code below to understand exactly what we did

In [3]:
# Merge caracteristics and places on 'Num_Acc'
data = cara.merge(places, on='Num_Acc')

In [4]:
# Create a common key to merge users amd vehicles on
users['Num_Acc_num_veh'] = users['Num_Acc'].map(lambda x: str(x)) + users['num_veh']
vehicles['Num_Acc_num_veh'] = vehicles['Num_Acc'].map(lambda x: str(x)) + vehicles['num_veh']
# Remove useless columns
vehicles = vehicles.drop(columns=['index'])
users = users.drop(columns=['index', 'Num_Acc', 'num_veh'])
# Merge vehicles and users
tmp = vehicles.merge(users, on='Num_Acc_num_veh', how='inner')

In [5]:
# Merge all datasets on 'Num_Acc'
data = data.merge(tmp, on='Num_Acc', how='inner')
del tmp

In [6]:
del cara
del places
del users
del vehicles

# Preprocessing

We will apply some preprocessing methods like standardization or missing values removal or imputing.
Remember to look at `challenge_variable.md` for a description of features.

## Clean Dataset

In [7]:
# drop lines without targets (if any)
data_cleaned = data[~np.isnan(data.grav)]

In [8]:
# Check whih features with highest ratio of NaN per column
(data_cleaned.isna().sum() / data_cleaned.shape[0]).sort_values(ascending=False)

v2                 0.953958
lat                0.632648
long               0.632648
gps                0.631134
pr1                0.463472
pr                 0.462609
v1                 0.383683
adr                0.184124
voie               0.048149
place              0.032898
secu               0.011189
lartpc             0.003175
larrout            0.002416
an_nais            0.001841
nbv                0.001731
vosp               0.001060
actp               0.000882
locp               0.000867
etatp              0.000825
infra              0.000662
env1               0.000653
plan               0.000445
prof               0.000443
surf               0.000427
obs                0.000427
circ               0.000392
situ               0.000355
obsm               0.000236
trajet             0.000186
manv               0.000106
choc               0.000064
atm                0.000039
col                0.000017
com                0.000005
catr               0.000002
hrmn               0

In [9]:
# Remove too incomplete features
too_incomplete_features=[
    'locp', 'actp', 'etatp'
]

In [10]:
# Remove features that can be safely considered useless for the predictive power of our model
useless_features=[
    'v2', 'lat', 'long', 'gps', 'pr1', 'pr', 'v1', 'adr', 'voie',
    'index_x', 'Num_Acc', 'Num_Acc_num_veh', 'Num_Acc', 'num_veh', 'index_y',
    'jour', 'an',
    'dep', 'com', 'env1',
]

In [11]:
data_cleaned.drop(columns=too_incomplete_features+useless_features, inplace=True)

In [12]:
data_cleaned.shape

(1209362, 32)

In [13]:
data_cleaned.dropna().shape

(1160867, 32)

In [14]:
del data
del useless_features
del too_incomplete_features

We now have a `data_cleaned` dataset! Let's now engineer our features as needed

## Prepare features and target

### Numerical features

In [15]:
features_numerical = ['lartpc', 'larrout', 'occutc', 'an_nais', 'nbv']

In [16]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, StandardScaler

def preprocess_numerical_features(X):
    '''
    Returns a new DataFrame with
    - Missing values replaced by Column Mean
    - Features Standard Scaled
    - Original Features names kept in the DataFrame
    '''
    data_num = pd.DataFrame.copy(X)
    imputer = SimpleImputer(strategy = 'mean')
    data_num[features_numerical] = imputer.fit_transform(data_num[features_numerical])
    scaler = StandardScaler()
    data_num[features_numerical] = scaler.fit_transform(data_num[features_numerical])
    return data_num

In [17]:
# Check your code below
preprocess_numerical_features(data_cleaned[features_numerical])

,lartpc,larrout,occutc,an_nais,nbv
0,-0.246173,-0.495102,-0.0581,1.273883,-0.052801
1,-0.246173,-0.495102,-0.0581,-0.224675,-0.052801
2,-0.246173,-0.138682,-0.0581,-0.835199,-0.052801
3,-0.246173,-0.138682,-0.0581,-1.279217,-0.052801
4,-0.246173,-0.138682,-0.0581,-1.168212,-0.052801
...,...,...,...,...,...
1209357,-0.246173,1.333487,-0.0581,0.052836,-1.248930
1209358,-0.246173,1.333487,-0.0581,1.051875,-1.248930
1209359,-0.246173,1.333487,-0.0581,-0.335680,-1.248930
1209360,0.659032,0.171249,-0.0581,-1.112710,-0.052801


❓ Do you get a Warning "A value is trying to be set on a copy of a slice from a DataFrame"?
If so, it may be because you are trying to modify the input DataFrame `data_cleaned`!

Read this [important blog on copy vs. view](https://www.practicaldatascience.org/html/views_and_copies_in_pandas.html) of pandas DataFrame and try to solve your warning by yourself



<details>
    <summary>Hint</summary>

`pd.DataFrame.copy()`
</details>

### (optional) cyclical features

In [18]:
# Do you see any cyclical features to process specifically?
# This can be done after a first baseline model is created.
features_cyclical = ['hrmn','mois']

In [19]:
import math
def preprocess_cyclical_features(X):
    '''
    Input: DataFrame X
    Output: Returns new DataFrame, where all its features X_i have been replaced
    by both their sin(2*Pi / cycle_length * X_i) and cos(2*Pi / cycle_length * X_i), and delete initial feature X_i.
    '''
    list_cols = X.columns
    new_X = X.copy()
    for feature, cycle_length in zip(features_cyclical,[12,30]):
        new_X[feature+'_sin'] = X[feature].map(lambda x: math.sin(2*math.pi / (x * cycle_length)))
        new_X[feature+'_cos'] = X[feature].map(lambda x: math.cos(2*math.pi / (x * cycle_length)))
    new_X.drop(columns=list_cols, inplace=True)
    return pd.DataFrame(new_X)

In [20]:
preprocess_cyclical_features(data_cleaned)

,hrmn_sin,hrmn_cos,mois_sin,mois_cos
0,0.004028,0.999992,0.019039,0.999819
1,0.004028,0.999992,0.019039,0.999819
2,0.000457,1.000000,0.207912,0.978148
3,0.000457,1.000000,0.207912,0.978148
4,0.000457,1.000000,0.207912,0.978148
...,...,...,...,...
1209357,0.000349,1.000000,0.017452,0.999848
1209358,0.000349,1.000000,0.017452,0.999848
1209359,0.000349,1.000000,0.017452,0.999848
1209360,0.000249,1.000000,0.017452,0.999848



### Categorical features

❓ Create the last group of feature (categorical features) without hardcoding them manually. Then, create the associated preprocessing method

In [21]:
features_categorical = list(set(data_cleaned.columns) - set(features_numerical) - set(features_cyclical) - {'grav'})

In [22]:
def preprocess_categorical_features(X):   
    ''' Returns a new DataFrame with dummified columns'''
    tmp = pd.DataFrame.copy(X.dropna().astype(int))
    tmp = pd.get_dummies(tmp, columns = features_categorical)
    return tmp

In [23]:
from sklearn.preprocessing import OneHotEncoder
def preprocess_categorical_features2(X):   
    ''' Returns a new DataFrame with dummified columns'''
    tmp = pd.DataFrame.copy(X.dropna().astype(int))
    oht = OneHotEncoder(handle_unknown='ignore')
    tmp = oht.fit_transform(tmp[features_categorical])
    return tmp

In [24]:
preprocess_categorical_features(data_cleaned[features_categorical])

,obsm_0,obsm_1,obsm_2,obsm_4,obsm_5,obsm_6,obsm_9,int_0,int_1,int_2,...,surf_0,surf_1,surf_2,surf_3,surf_4,surf_5,surf_6,surf_7,surf_8,surf_9
0,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209357,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1209358,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1209359,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1209360,1,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


❓ Create the new `data_preprocessed` dataset by concatenating all three preprocessing, and then drop all remaining NaN that could not have been handled previously despite our preprocessing

In [25]:
data_preprocessed = pd.merge(preprocess_numerical_features(data_cleaned[features_numerical]),\
                             preprocess_categorical_features(data_cleaned[features_categorical]),\
                             left_index=True, right_index=True)

In [26]:
data_preprocessed.head(5)

,lartpc,larrout,occutc,an_nais,nbv,obsm_0,obsm_1,obsm_2,obsm_4,obsm_5,...,surf_0,surf_1,surf_2,surf_3,surf_4,surf_5,surf_6,surf_7,surf_8,surf_9
0,-0.246173,-0.495102,-0.0581,1.273883,-0.052801,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,-0.246173,-0.495102,-0.0581,-0.224675,-0.052801,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,-0.246173,-0.138682,-0.0581,-0.835199,-0.052801,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,-0.246173,-0.138682,-0.0581,-1.279217,-0.052801,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,-0.246173,-0.138682,-0.0581,-1.168212,-0.052801,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [27]:
data_preprocessed = pd.merge(preprocess_cyclical_features(data_cleaned),data_preprocessed,\
                             left_index=True, right_index=True)

In [28]:
data_preprocessed.shape

(1166887, 234)

In [29]:
data_preprocessed = pd.merge(data_cleaned[['grav']],data_preprocessed,\
                             left_index=True, right_index=True)

In [30]:
data_preprocessed.shape

(1166887, 235)

In [31]:
del data_cleaned

## Split Dataset
❓ Create X and y, and don't forget to convert the classification into a binary task.

For instance:

```python
data['grav_binary'] = data['grav'].replace({1: 0, 4: 0, 2: 1, 3: 1})
```

In [32]:
data_preprocessed_s = data_preprocessed.sample(10000)
data_preprocessed_s.shape

(10000, 235)

In [33]:
# X = data_preprocessed.drop(columns = ['grav'])
y = data_preprocessed['grav'].replace({1: 0, 4: 0, 2: 1, 3: 1})
y.shape

(1166887,)

In [34]:
X_small = data_preprocessed_s.drop(columns = ['grav'])
y_small = data_preprocessed_s['grav'].replace({1: 0, 4: 0, 2: 1, 3: 1})

In [35]:
# Train Test Split both datasets
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
Xs_train, Xs_test, ys_train, ys_test = train_test_split(X_small, y_small, test_size = 0.3)

In [36]:
# (optional) Create here a train/eval split within the train set itself.
# Some powerfull models (XGBOOST, Neural Network...) which are prone to overfitting on the traning set, needs "early stopping criteria", to avoid descending the gradient completely and avoid overfitting.

# Features exploration

You now have a dataset ready for training! 
**Skip directly to section 5 to get a baseline model working ASAP**, and only then come back to this section 4 if you want to better understand your X and get inspiration for the best model to use, or for some feature selection to reduce model complexity

## Visualization

❓Investigate your X. Are features strongly correlated? Are some feature more important than other?

In [ ]:
# data_preprocessed.corr()

## PCA

❓Fit a PCA and plot the cumulated sum of explained variance ratio of your Principal Component. Do you see any clear elbow?

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X_small, y_small)
var = pca.explained_variance_ratio_

In [ ]:
import seaborn as sns
kwargs = {'cumulative': True}
sns.distplot(var, hist_kws=kwargs, kde_kws=kwargs)

## Forest-based most important features

❓ Fit a default RandomForestClassifier on a small smaple to estimate the top 20 feature importance. Do they make intuitive sense to your point of view?  Do you see any clear elbow for dimension-reduction?

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

forest = RandomForestClassifier()

forest_cv = cross_validate(forest, 
                            X_small,
                            y_small,
                            scoring = "f1_macro",
                            cv=5)

forest_cv['test_score']
forest.fit(Xs_train, ys_train)

RandomForestClassifier()

In [38]:
forest_results = pd.DataFrame({'feature': X_small.columns,'importance': forest.feature_importances_})
forest_results.sort_values(by='importance', ascending = False).head(20)
top20 = forest_results.sort_values(by='importance', ascending = False).head(20)
top20features = top20['feature'].values
top20features

array(['an_nais', 'hrmn_sin', 'larrout', 'mois_sin', 'mois_cos', 'nbv',
       'secu_11', 'obs_0', 'catr_3', 'agg_2', 'secu_21', 'catv_7',
       'lartpc', 'agg_1', 'catr_4', 'choc_1', 'trajet_5', 'manv_1',
       'circ_2', 'col_1'], dtype=object)

In [39]:
data_top20 = data_preprocessed[top20features]

❓ (Optional) There are better ways to estimate feature importance in a RandomForest. Feel free to try to two following options

**Option 1** : Recursive-method
1. Train a first model, note top1 feature (computed based on the gini-explicative power of the feature, in each tree)
2. Remove top1 from your X and retrain a RandomForest. Note top1 feature and it's relative importance
3. Loop

**Option 2** : Permutation-method ([sklearn.inspection.permutation_importance](https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance)), works with any model!
1. Train a first model, keep track of its accuracy
2. Take one feature and shuffle its columns. Compute new accuracy of the corrupted dataset, and note by how much it has been reduced.
3. Loop over all features and rank them by accuracy reduction

In [ ]:
# YOUR CODE HERE

# Modeling

## Baseline performance metrics

❓ What is the class balance of your target?  
Do you think acccuracy would be a good score?
If you don't want to favor any class over the other, what would be a good performance metric for your problem? 

In [ ]:
baseline = y.value_counts()[1]/(y.value_counts()[1] + y.value_counts()[0])
baseline

In [ ]:
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
dum = DummyClassifier(strategy='most_frequent')
dum.fit(Xs_train, ys_train)
dum.score(Xs_test, ys_test)
print(classification_report(ys_test, dum.predict(Xs_test)))

In [ ]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(dum, X_small, y_small, scoring = 'f1_macro', cv = 5)
print(cv_results["test_score"].mean())

<details>
    <summary>Answer</summary>

In such an unbalanced problem, accuracy is meaningless: A very dumb model predicting always zeros would have great accuracy, to the detriment of the predictive power of class  1, which has precision and recall equal to zero!
    
The non-weighted mean between both f1 score of each class called `f1_macro` would be a good measure for this type of problem.
</details>

## Simple Model (A first iteration)

❓ Create a simple model, fast to train, to classify the severity of the accidents. Start simple. Don't forget to fit on your training set and evaluate the score on your test set. Can you beat the Baseline? What about its `f1_macro` score? Measure the time it takes on the full dataset, with `%%time` 

In [ ]:
# %%time
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_validate
# log = LogisticRegression(max_iter = 1000)
# cv_results = cross_validate(log, X_small, y_small, scoring = 'f1_macro', cv = 5)
# print(cv_results["test_score"].mean())

In [ ]:
# %%time
# from sklearn.neighbors import KNeighborsClassifier 
# from sklearn.model_selection import cross_validate
# knn = KNeighborsClassifier()
# cv_results = cross_validate(knn, X_small, y_small, scoring = 'f1_macro', cv = 5)
# print(cv_results["test_score"].mean())

# 🔥🔥🔥 Advanced Models - LeWagon batch contest ! 🔥🔥🔥

❓ Now it's your turn to shine! Play with different models and try to find the best one on your training set!
- Send your best `f1_macro` score (as defined above) to your slack channel without saying which model you used!
- ⚠️ Only send score tested on a full `y_test` of complete size (30% of 1M rows!) will be taken into account
- Feel free to use your X_small for investigation purpose
- If it takes too long to train, simplify your model, or use better feature preprocessing/selection

The winner will present its notebook to the class during the reboot 💪

(Don't forget, your Notebook should be made to be run from top to bottom in one go!)

<details>
    <summary>Some hints</summary>
Take a closer look at feature engineering: Are there some features we haven't correctly preprocessed?  
    
Most of the time, a good dataset trumps a good model!
</details>

In [40]:
del data_preprocessed
del data_preprocessed_s

In [42]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
log = LogisticRegression(max_iter = 200, C=3, class_weight = 'balanced')
cv_results = cross_validate(log, data_top20, y, scoring = 'f1_macro', cv = 3, n_jobs = -1)
print(cv_results["test_score"].mean())

CPU times: user 98 µs, sys: 0 ns, total: 98 µs
Wall time: 106 µs


In [43]:
%%time
from sklearn.ensemble import VotingClassifier


ensemble = VotingClassifier([('rf', forest), ('lr',log)])

ensemble_cv = cross_validate(ensemble, 
                            data_top20,
                            y,
                            scoring = "f1_macro",
                            cv=3,
                            n_jobs = -1)

ensemble_cv['test_score'].mean()

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

### (Optional) - Pipeline most steps (prepross & fit) in one single Sklearn Pipeline